- 01_matmul
- 02_fully_connected
- 02a_why_sqrt5
- 02b_initialization
- 03_minibatch_training

Table of contents
1. Initial Setup
2. Basic training Loop
3. Using parameters and optim
4. Dataset and DataLoader
5. Validation

# 1. Initial Setup

## Data

### Fastai for Colab env

In [1]:
!git clone http://github.com/fastai/course-v3.git

Cloning into 'course-v3'...
remote: Enumerating objects: 5498, done.
remote: Total 5498 (delta 0), reused 0 (delta 0), pack-reused 5498
Receiving objects: 100% (5498/5498), 257.94 MiB | 34.24 MiB/s, done.
Resolving deltas: 100% (2992/2992), done.


In [2]:
%cd course-v3/nbs/dl2

/content/course-v3/nbs/dl2


In [3]:
from exp.nb_02 import *

In [4]:
!cat exp/nb_02.py


#################################################
### THIS FILE WAS AUTOGENERATED! DO NOT EDIT! ###
#################################################
# file to edit: dev_nb/02_fully_connected.ipynb

from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

from torch.nn import init

def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

from torch import nn

### import data

In [4]:
train_x, train_y, valid_x, valid_y = get_data()

In [6]:
train_x.shape, train_x.type()

(torch.Size([50000, 784]), 'torch.FloatTensor')

In [5]:
m, n_in = train_x.shape
nh = 70 # on perpose changed little bit bigger
n_out = train_y.max().item()+1

In [7]:
# n_in, nh, n_out = train_x.shape[1], 32, train_y.max().item()+1; n_in, nh, n_out

(784, 32, 10)

[^1]

In [11]:
n_in, nh, n_out

(784, 70, 10)

In [6]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x
    # def forward(self, x, y):
    #     for l in self.layers: x = l(x)
    #     return(self.loss(x, y))

In [55]:
model = Model(n_in, nh, n_out)

In [8]:
y_hat = model(train_x)

## Loss Function

### Softmax

* formula of softmax : $f(x) = \frac{exp(x)}{\sum exp(x_i)}$
* We need softmax at last layer since we have to map it to the probabilistic space
* code of softmax is below

In [15]:
def softmax(x): return x.exp() / x.exp().sum(-1, keepdim=True)
# def softmax(x): return x.exp() / x.exp().sum()

[^2]

In [16]:
p = softmax(y_hat); p[0].sum()

tensor(1., grad_fn=<SumBackward0>)

- I need *log* of softmax because we will use cross-entropy which has form of $H(X, q) = H(X) + D(p|q) = - \sum_{x} p(x)\  log\ q(x)$ [^3]

In [17]:
def log_softmax(x): return (x.exp() / x.exp().sum(-1, keepdim=True)).log()

In [18]:
a = log_softmax(y_hat)

### Cross Entropy

- see the footnote 3 regarding further information of cross entropy
- One possible reason is because an information theory is based on descrete and determinate value

### Negative log likelihood function

In [19]:
def nll(x, y): return -x[range(x.shape[0]), y].mean() #x: prediction, y:target 

In [20]:
nll(y_hat, train_y)

tensor(-0.0208, grad_fn=<NegBackward>)

- Negative value is related to entropy, roughly variable which is rare is meaningful [^4]

### LogSumExp

- what is logsumexp?

$log \frac{e^{x}}{\sum_{i} e^{x_i}} \rightarrow \log e^{x} - \log \sum_{i} e^{x_i} \rightarrow x - \log \sum_{i} e^{x_i}$

In [21]:
def log_softmax(x): return x - x.logsumexp(-1, keepdim=True)

In [22]:
fn = lambda x: x - x.sum(-1, keepdim=True)

In [23]:
test_near(log_softmax(y_hat), fn(y_hat)) #[^5]

AssertionError: ignored

# 2. Basic Training Loop

### 1.
 Write down the procedure the training loop repeats (4 steps)

1. make a prediction
2. get loss from output
3. gradient from output
4. update gradient



### 2.

In [9]:
def accuracy(hat, trg): return (hat.max(-1).indices == trg).sum() / float(hat.shape[0]) # made f1 score, but had to see if this is right.

In [10]:
accuracy(y_hat, train_y)

tensor(0.1112)

In [11]:
def acc(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

In [12]:
acc(y_hat, train_y)

tensor(0.1112)

3.

In [14]:
bs = 64

In [56]:
xb, yb = train_x[0:bs], train_y[0:bs]
accuracy(model(xb), yb)

tensor(0.1094)

### 4

In [19]:
import torch.nn.functional as F
loss_fun = F.cross_entropy
epochs = 1

In [17]:
lr = 0.5

When I have no attribute checker

In [30]:
[l.weight.grad for l in model.layers]

AttributeError: ignored

In [31]:
[l.weight.grad for l in model.layers if hasattr(l, 'weight')]

[None, None]

In [44]:
model = Model(n_in, nh, n_out)
yb_hat = model(xb)
loss = loss_fun(yb_hat, yb)
loss.backward()
loss, accuracy(model(xb), yb)

(tensor(2.3024, grad_fn=<NllLossBackward>), tensor(0.1094))

In [ ]:
[l.weight.grad for l in m.layers if hasattr(l, 'weight')]

In [59]:
def fit(m, **kargs):
    for epoch in range(epochs):
        # for i in range((train_x.shape[0] // bs)+1):
        for i in range((train_x.shape[0]-1)//bs +1):
            # [^10]
            xb, yb = train_x[(bs*i):(bs*(i+1))], train_y[(bs*i):(bs*(i+1))]
            loss = loss_fun(m(xb), yb)
            loss.backward()
            with torch.no_grad():
                for l in m.layers: # [^8]
                    if hasattr(l, 'weight'): #[^6]
                        l.weight -= l.weight.grad * lr
                        l.bias -= l.bias.grad * lr
                        l.weight.grad.zero_() #[^7]
                        l.bias.grad.zero_()
    return m

In [60]:
m1 = fit(Model(n_in, 50, n_out))

In [61]:
loss_fun(m1(xb), yb), accuracy(m1(xb), yb)

(tensor(0.1438, grad_fn=<NllLossBackward>), tensor(0.9531))

[^9]

# Notes

[^1]: I don't have to define parameters no more, but obviously at first I tried to iniailized Model with `w1, b1, ....`, meaning I didn't practice enough the previous part, part of the torch nn, where I get parameters

[^2]: be cautious since `train_x.sum(-1)` will squeeze the rank, so that you need arg `keepdim:bool`, false as default

[^3]: see *eq (2.46)*, Foundations of Natural Language Processing, Christopher D. Manning and Hinrich Schütze

[^4]: Should check for `cross-entropy $\approx$ softmax of negative likelihood`

[^5]: why I can't make it like `fn`?

[^6]: We need this since not 'all' the layers have weight, in this care `ReLU`

[^7]: Still can't get why I should do this

[^8]: Basically, this is model instance which comes from `Model()` inheriting `nn.Module`.</br> `layers` is attribute of that instance, not a method!

[^9]: (solved) I cant' get any difference between jeremy's and mine, I just changed node size to 70 (50 at Jeremy) But, does that make this much diffference????????<br/> >>> I did a mistake which take gradients out from gradient!!!!!!!you should take out from variable, not gradients... I wasted almost an hour

[^10]: `cliping batchsize` part is somewhat different from jeremy's